# Function basis methods

There are many reasons why nonlinear numerical relativity is so expensive. The choice of numerical method is one that we can actually control. This links to the idea of *efficiency*: the computational cost of a method to reach a given accuracy. This is heavily problem dependent, but some general points can be made.

So far we have looked at finite difference and finite volume methods which converge *polynomially*, as $\mathcal{O}(\Delta x^p)$. As the computational cost scales as $N^4$ and $N \sim \Delta x^{-1}$, only high order methods are practical when high accuracy (for phasing error, particularly for next generation detectors) is needed.

It is possible to do better. However, the setup cost (in both mental and computational effort) is larger, and the more complex methods have their own problems.

## Spectral methods

For finite difference methods we assumed the function $q(x)$ was note at specific points $x_i$, giving $q_i$, and fitted an interpolating function, usually a low order polynomial $g(x)$, through those points. We then approximated the derivative at the grid points using the derivative of $g$.

In spectral methods we approximate the function using a function basis, usually using orthogonal functions $\phi_n(x)$, so that
$$
q(x) = \sum_n c_n \phi_n(x) \, .
$$
If we set the basis to be the complex exponentials, $\phi_n(x) = \exp(i n x)$, then this is a Fourier series. Other standard choices for the basis would be Chebyshev polynomials or Legendre polynomials.

The point of using function basis methods is the expectation that they will converge faster than finite difference methods. We know from Fourier series that, for smooth functions, the approximation error converges exponentially with the number of terms in the series. The hope is that this rapid convergence will carry over when applied to PDEs.

At this point the solution is exact (provided that $q$ satisfies any convergence theorem requirements, which often include differentiability). However, it is not useful for numerics, as there will typically be infinitely many terms in the sum. To use this approach we need to truncate the expansion to a finite number of terms $N$, and plug it into the evolution equations.

### Burgers equation

As an example we will look at Burgers equation
$$
\partial_t q + q \partial_x q = 0
$$
on $x \in [0, 2 \pi]$ with periodic boundary conditions. We will use the Fourier basis, so that
$$
q(x, t) = \sum_{n=-N}^N c_n(t) \exp(i n x) \, .
$$
We can then plug this into the equation. This gives
$$
\sum_{n=-N}^N \left( \partial_t c_n(t) \right) \exp(i n x) + \sum_{m=-N}^N c_m(t) \left( \sum_{k=-N}^N i k c_k(t) \exp(i (k + m) x) \right) = 0 \, .
$$
Gathering exponential terms we get
$$
\sum_{n=-N}^N \exp(i n x) \left[ \partial_t c_n(t) + \sum_{k+m=n} i k c_m(t) c_k(t) \right] = 0 \, .
$$
Applying the standard orthogonality relations we get the evolution equations for the coefficients,
$$
\partial_t c_n(t) + \sum_{k+m=n} i k c_m(t) c_k(t) = 0 \, .
$$
This is $2N+1$ evolution equations for the $2N+1$ complex coefficients $c_n(t)$.

Note that this system is only closed if we ignore the fact that the nonlinear product could, in principle, couple wave numbers outside our truncated range. For now we ignore this point; however, correctly dealing with this is important for ensuring the long term accuracy of a spectral method.

### Implement and check

We will use the Method of Lines, with the time evolution provided by `scipy` to get high accuracy. We will evolve a single sine wave forward in time.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import ipympl
from scipy.integrate import ode
from scipy.optimize import root_scalar


In [ ]:
def initial_data(N):
    c = np.zeros(2*N+1, dtype=np.complex64)
    c[N-1] = 0.5*1j
    c[N+1] = -0.5*1j
    return c

def get_q(c, x):
    N = (len(c)-1)//2
    q = np.zeros_like(x)
    for i in range(2*N+1):
        q += np.real(c[i]*np.exp(1j*(i-N)*x))
    return q

def spectral(t, c):
    N = (len(c)-1)//2
    dcdt = np.zeros_like(c)
    raise NotImplementedError("The spectral RHS needs implementing")
    # I would recommend that the following loop structure is used
    # The print statement is left in for debugging
    # Note that the loop assumes all mode indexes run -N, ..., N
    # But Python index means the c array has indexes 0, ... 2 N + 1
    # for n in range(-N, N+1):
    #     for k in range(-N, N+1):
    #         m = n - k
    #         if (abs(m) <= N):
    #             # print(t, n, m, k, c[m+N], c[k+N])
    #             # Take into account that spectral indexing and
    #             # internal Python indexing are offset by N
                
    return dcdt

# This uses scipy do do the method of lines
# This should make the time integrator error extremely small
def evolve_scipy(N, t_end=0.5, dtfac=0.1):
    c = initial_data(N)
    
    dt = dtfac * 1/len(c)**2
    r = ode(spectral).set_integrator('zvode', max_step=dt)
    r.set_initial_value(c)
    while r.successful() and r.t < t_end:
        if r.t + dt > t_end:
            dt = t_end - r.t
        r.integrate(r.t + dt)
    return r.y


We will also use the method of characteristics to compute the exact solution, which works fine up until characteristic breaking time $t \simeq 1$.

In [ ]:
def exact_root(q, x, t):
    return q - np.sin(x - q * t)

def exact(xx, t):
    q_exact = np.zeros_like(xx)
    for i, x in enumerate(xx):
        sol = root_scalar(exact_root, args=(x, t), bracket=[-1, 1])
        q_exact[i] = sol.root
    return q_exact

In [ ]:
# This does the evolution and plots the result at t=0.5
x = np.linspace(0, 2*np.pi, 100)
N = 5
t_end = 0.5
c = evolve_scipy(N, t_end=t_end)
plt.figure()
plt.plot(x, get_q(c, x), label=rf"$N={N}$")
plt.plot(x, exact(x, t_end), 'r--', label='exact')
plt.legend()
plt.title(rf'$t={t_end:.1f}$')
plt.xlabel(r'$x$')
plt.ylabel(r'$q(x)$')
plt.xlim(0, 2*np.pi)
plt.show()

We see the solution is very accurately captured. Now check the convergence rate:

In [ ]:
Ns = 2*np.arange(3, 9)
errors = np.zeros(len(Ns))
for i, N in enumerate(Ns):
    c = evolve_scipy(N, t_end=t_end)
    q = get_q(c, x)
    q_exact = exact(x, t_end)
    errors[i] = np.linalg.norm(q - q_exact)

In [ ]:
plt.figure()
plt.semilogy(Ns, errors, 'kx', label="Spectral errors")
p = np.polyfit(Ns, np.log(errors), 1)
plt.semilogy(Ns, np.exp(p[1]+p[0]*Ns), 'b--', label=rf"$\propto \exp({p[0]:.2f}N)$")
plt.legend()
plt.xlabel(r'$N$')
plt.ylabel("Error")
plt.show()

Exponential convergence, as we wanted. This is *so* much more efficient than polynomial scaling methods. As a sketch plot, assume that methods order one, two, and four happen to have the same error when $N=6$ as here. Then we can plot how the errors would go:

In [ ]:
plt.figure()
Ns = np.linspace(6, 40)
plt.semilogy(Ns, np.exp(p[1]+p[0]*Ns), 'b--', label=rf"$\propto \exp({p[0]:.2f}N)$")
plt.semilogy(Ns, np.exp(p[1]+p[0]*Ns[0])*(Ns/Ns[0])**(-1), 'r--', label=r"$\propto N^{-1}$")
plt.semilogy(Ns, np.exp(p[1]+p[0]*Ns[0])*(Ns/Ns[0])**(-2), 'g--', label=r"$\propto N^{-2}$")
plt.semilogy(Ns, np.exp(p[1]+p[0]*Ns[0])*(Ns/Ns[0])**(-4), 'k--', label=r"$\propto N^{-4}$")
plt.legend()
plt.xlabel(r'$N$')
plt.ylabel("Error")
plt.show()

The huge difference in accuracy is clear.

### Problems with spectral methods

#### Dealing with nonlinearity

The nonlinear term $q \partial_x q$ was manipulated into a specific form that couples (in a relatively straightforward way) different Fourier modes. This is not possible in general, although the mode coupling is generic.

The standard approach to dealing with this is to use a *pseudo-spectral method*. The truncated basis coefficients are equivalent to representing the function values on a grid. This means that we compute the nonlinear term in real space, and then transform it back to compute the coefficients. For numerical stability this grid is typically *not* uniformly spaced. The cost of transforming between real space and frequency space reduces the efficiency of the method, but does not dominate the cost.

#### Time stepping

The standard Courant limit restricts the timestep to be inversely proportional to the grid spacing. When the spectral method is phrased in terms of frequency space there is no grid spacing, so the limiting timestep is unclear. When using a pseudo-spectral method with an uneven grid, the link to the *smallest* grid spacing is clearer, but still not fully transparent. The key point result of more detailed stability calculations is that the timestep goes inversely with the *square* of the number of modes. This significantly reduces the timestep, making higher accuracy much more expensive and increasing accumulated errors from the time integrators.

It is still typically true that spectral methods are more efficient than (say) finite difference methods, but the difference is not as large as initial expectations would suggest.

#### Discontinuities

Fourier series famously exhibit *Gibb's oscillations*: under/over-shoots when representing discontinuities, whose maximum error does not converge as the number of coefficients increases. More generally, any loss of differentiability in the continuum solution makes the spectral representation converge polynomially (just like a finite difference method), with the convergence order linked to the smoothness of the function.

Numerical methods will be directly affected whenever the function being approximated is steep enough to appear to lose smoothness. As a lot of matter models, particularly fluids, will naturally steepen gradients through (eg) acoustic modes, this is a real problem.

The current consensus is that spectral methods should not be used for matter simulations, or at least not for the matter sector. Some use spectral methods for the spacetime sector and finite volume methods on a separate grid for the matter sector. In principle shocks in the matter sector should lead to a loss of smoothness in the spacetime; however, the impact of this has not been seen. This is a very complex approach.

#### Parallel efficiency

With finite difference methods, the approximate derivative at a single point depends on a small number of its neighbours. When using parallelisation to speed up the simulation, only these small number of neighbours need to be communicated between compute cores.

With a spectral method, the function at any one point depends on *all* the coefficents. Equivalently, in a pseudo-spectral method its derivative depends on the values at *all* the collocation points. This makes parallelisation much more complicated to implement and much less efficient.

The usual *hope* is that the huge improvement in the convergence rate is sufficient to reduce the need for parallel computing in the first place. This is typically not sufficient for full NR simulations, however, so some parallelistation is still needed.

The standard approach is to use *spectral elements*. The domain is split into a small number of elements and *separate* spectral expansions are used in each element. Then the "only" thing that needs communicating is the function values at the element boundaries. This helps significantly, but good choices of elements can involve a lot of trial and error.

#### Boundaries

Spectral methods can be *remarkably* touchy when applied to domains with complex boundaries. In general this is not a problem in NR - asymptotic flatness and few relevant "interior" solid boundaries see to this. However, there are a range of cases where features *act like* boundaries: spectral element methods, neutron star surfaces, and in black hole singularity excision. Many of the problems are made worse as these features are typically dynamic, so the boundary moves. 

The usual solution is to work with spectral elements where each element can be mapped to a simple shape cube or a sphere. Minimizing the distortion of the map helps stability. Again, there is trial and error involved in making this work.

# Discontinuous Galerkin methods

Discontinuous Galerkin (DG) methods are a type of spectral element method. They split the domain into elements (like the cells in finite volume methods), and use a function basis inside each element. The main point is that the values of the function at the element boundaries are not required to match. This allows for discontinuities in the solution (in principle!), and is particularly nice with conservation laws. *In principle* they combine flexibility with efficiency. As each element communicates very little with its neighbours (even less than high order finite difference methods), they are very parallelisable. As the number of basis functions within a single element can be increased, in smooth regions they can achieve spectral convergence. As each element is (quasi-)independent they can have different sizes, effectively giving mesh refinement.

The complexity of the setup of a DG method is, however, high. As yet they have neither the efficiency on smooth solutions of spectral methods, nor the robustness of finite differences and volumes, particularly on discontinuous flows. Their popularity in classical CFD contexts may lead to them being the method of the future in NR, however.

## Theory

Take a conservation law (for example) of the form
$$
\partial_t q + \partial_x f(q) = 0 \, .
$$
Multiply by a *test function* $w(x)$ and integrate over an element $V_i$, getting
$$
\int_{V_i} w \partial_t q + \int_{V_i} f(q) \partial_x w = -\left[ w f(q) \right]_{V_i} \, .
$$
Note that we have integrated by parts to switch the spatial derivative from the flux function to the test function. As we have not assumed the function is continuous at the element boundary (the "Discontinuous" part), we pick up a boundary term due to the boundary flux jump, which is moved to the right hand side.

Now assume an orthogonal basis $\phi_n(x)$ for all functions we consider (this is the "Galerkin" part) and write
$$
\begin{aligned}
q = \sum_n q_n(t) \phi_n(x) \, , \\
w = \sum_k w_k \phi_k(x) \, .
\end{aligned}
$$
Plugging this into the equation of motion gives
$$
\sum_k w_k \left\{ \sum_n \partial_t q_n \int_{V_i} \phi_n \phi_k + \sum_n f(q_n) \int_{V_i} \phi_n \partial_x \phi_k \right\} = -\left[ w f(q) \right]_{V_i} \, .
$$
As the test function is arbitrary we get equations of the form
$$
M \partial_t \mathbf{q} + S^T \mathbf{f}(\mathbf{q}) = -\mathbf{F} \, ,
$$
where the *mass matrix* $M$ and *stiffness matrix* $S$ are given by integrals over the element of the basis functions and their derivatives, and the *force* (or *flux*) vector corrects the first and last nodal values to account for the discontinuity at the element boundaries. The mass and stiffness matrices depend only on the choice of basis functions and the size of the element, so can be pre-computed for efficiency.

There are two function bases of interest. When considering modes it is typical to use Legendre polynomials $P_n(x)$. When considering nodes it is typical to use *indicator functions* - Lagrange polynomials that are 1 at a single node in the element and zero at all other nodes in the element. That means the coefficients for the basis functions are exactly the function values of the solutions at the nodes. The indicator functions are generally not orthogonal, so calculations requiring orthogonality are done using the modes, and nonlinear calculations (for example, of the flux function) are done using the nodes. The two bases are related by a Vandermonde matrix, which again can be pre-computed.

### Implementation

In [ ]:
from numpy.polynomial import legendre

# quadpy may require a licence. It's very useful if you can get one.
#import quadpy
# If not, use the hack below for the Gauss-Lobatto points.

def gl_nodes(m):
    nodes = np.zeros(m+1)
    nodes[0] = -1
    nodes[-1] = 1
    # p is the Legendre polynomial P_m
    p = np.zeros(m+1)
    p[-1] = 1
    # The internal Gauss-Lobatto nodes are the roots of P'_{m}
    nodes[1:-1] = legendre.legroots(legendre.legder(p))

    return nodes

def flux(q):
    return q**2/2

def dg_solve(m, Ne, t_end=0.1):
    # If you have a licence for quadpy this is more accurate, especially for large m
    # GL = quadpy.c1.gauss_lobatto(m+1)
    # nodes = GL.points
    nodes = gl_nodes(m)
    # To go from modal to nodal we need the Vandermonde matrix
    V_hat = legendre.legvander(nodes, m)
    c = np.eye(m+1)
    # Orthonormalize
    for p in range(m+1):
        V_hat[:, p] /= np.sqrt(2/(2*p+1))
        c[p, p] /= np.sqrt(2/(2*p+1))
    d_V_hat = legendre.legval(nodes, legendre.legder(c)).T
    # The mass matrix comes from the Vandermonde matrix
    M = np.linalg.inv(V_hat @ V_hat.T)
    M_inv = V_hat @ V_hat.T
    # The stiffness matrix is also linked
    K = (M @ (d_V_hat @ np.linalg.inv(V_hat))).T
    # The above matrices are on a "reference" element.
    # The true matrices need scaling to the physical width,
    # which might vary element-to-element
    Minvs = []
    Ks = []
    all_nodes = np.zeros((m+1)*(Ne+2))
    dx_e = 1/Ne
    for e in range(Ne+2):
        Minvs.append(M_inv / dx_e * 2 )
        Ks.append(K)
        all_nodes[e*(m+1):(e+1)*(m+1)] = ((e-1)+(1+nodes)/2) * dx_e
    q0 = np.sin(2*np.pi*all_nodes)

    # A slight hack: define the RHS nested inside,
    # to give access to the mode number m without explicitly passing it through.
    # This means the RHS function can be used directly with scipy
    def rhs(t, q):
        dqdt_nodal = np.zeros_like(q)
        dqdt = np.zeros_like(q)
        # Loop over elements
        for e in range(1, Ne+1):
            lo = e*(m+1)
            hi = (e+1)*(m+1)
            dqdt_nodal[lo:hi] += Ks[e] @ flux(q[lo:hi])
            # LF, alpha = 1
            dqdt_nodal[lo] += 0.5 * (flux(q[lo]) + flux(q[lo-1]) - (q[lo] - q[lo-1]))
            dqdt_nodal[hi-1] -= 0.5 * (flux(q[hi]) + flux(q[hi-1]) - (q[hi] - q[hi-1]))
            dqdt[lo:hi] = Minvs[e] @ dqdt_nodal[lo:hi]
        # Periodic boundaries
        dqdt[:m+1] = dqdt[-2*(m+1):-(m+1)]
        dqdt[-(m+1):] = dqdt[(m+1):2*(m+1)]
        return dqdt

    cfl = 0.1/(2*m+1)
    # Again use scipy with high order integration
    r = ode(rhs).set_integrator('dopri5', max_step=cfl*dx_e)
    r.set_initial_value(q0)
    while r.successful() and r.t < t_end:
        if r.t + cfl*dx_e > t_end:
            r.integrate(t_end)
        else:
            r.integrate(r.t + cfl*dx_e)

    return all_nodes, r.y

fig, ax = plt.subplots(3, 1, figsize=(5, 11))

m = 4 # modes
Ne = 8 # Number of elements
t_end = 0.1
all_nodes, soln = dg_solve(m, Ne, t_end)
q0 = np.sin(2*np.pi*all_nodes)

ax[0].plot(all_nodes, q0, 'k--', label="Initial")
ax[0].plot(all_nodes, soln, 'b-x', label=rf"t={t_end:.1f}")
# Note the differing domains lead to the scaling here.
ax[0].plot(all_nodes, exact(2*np.pi*all_nodes, 2*np.pi*t_end), 'r--', label=rf"Exact t={t_end:.1f}")
ax[0].legend()
ax[0].set_xlabel(r"$x$")
ax[0].set_ylabel(rf"$q, m={m}, N={Ne}$")
ax[0].set_xlim(0, 1);

Ne = 20
ms = np.arange(2, 7)
errors = np.zeros(len(ms))
for i, m in enumerate(ms):
    all_nodes, soln = dg_solve(m, Ne, t_end)
    errors[i] = np.linalg.norm(soln - exact(2*np.pi*all_nodes, 2*np.pi*t_end)) # This drops the dx factor!
ax[1].semilogy(ms, errors, 'kx', label='Errors')
pp = np.polyfit(ms, np.log(errors), 1)
ax[1].semilogy(ms, np.exp(pp[1]+pp[0]*ms), 'b--', label=rf"$\propto \exp({pp[0]:.2f} m)$")
ax[1].set_xlabel(r"Modes $m$")
ax[1].set_ylabel(r"Error")
ax[1].set_xticks(ms)
ax[1].set_xticklabels(ms)
ax[1].legend()

m = 4
Nes = 2**np.arange(4, 9)
errors3 = np.zeros(len(Nes))
errors4 = np.zeros(len(Nes))
for i, Ne in enumerate(Nes):
    all_nodes3, soln3 = dg_solve(3, Ne, t_end)
    all_nodes4, soln4 = dg_solve(4, Ne, t_end)
    errors3[i] = np.linalg.norm(soln3 - exact(2*np.pi*all_nodes3, 2*np.pi*t_end))/np.sqrt(Ne)
    errors4[i] = np.linalg.norm(soln4 - exact(2*np.pi*all_nodes4, 2*np.pi*t_end))/np.sqrt(Ne)
ax[2].loglog(Nes, errors3, 'kx', label='Errors m=3')
ax[2].loglog(Nes, errors4, 'r+', label='Errors m=4')
pp3 = np.polyfit(np.log(Nes), np.log(errors3), 1)
pp4 = np.polyfit(np.log(Nes), np.log(errors4), 1)
ax[2].loglog(Nes, np.exp(pp3[1]+pp3[0]*np.log(Nes)), 'k--', label=rf"$\propto N^{{{pp3[0]:.2f}}}$")
ax[2].loglog(Nes, np.exp(pp4[1]+pp4[0]*np.log(Nes)), 'r--', label=rf"$\propto N^{{{pp4[0]:.2f}}}$")
ax[2].set_xlabel(r"Modes $m$")
ax[2].set_ylabel(r"Error")
ax[2].set_xticks(Nes)
ax[2].set_xticklabels(Nes)
ax[2].legend()

fig.tight_layout()
plt.show()

Note the uneven nature of the grid, which is natural for DG methods. The first simulation effectively uses 32 bits of information (8 elements, 4 modes/nodes per element) but still captures the solution well.

We see the expected result: by modifying the number of modes we get spectral convergence, and by modifying the number of elements we get polynomial convergence linked to the number of modes used.